In [30]:
import os
import csv
import cv2
import torch
import numpy as np
import pandas as pd
from collections import defaultdict

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
from ultralytics import YOLO
from scipy.spatial import distance
from collections import deque
import matplotlib.pyplot as plt

VIDEO_PATH = '/content/drive/MyDrive/Task 3/dataset_video.mp4'  # Your confirmed video path
DISTANCE_THRESHOLD = 150      # Distance in pixels to consider people "close"
PERSISTENCE_FRAMES = 3        # Number of consecutive frames with crowd to log
CONFIDENCE_THRESHOLD = 0.3    # Minimum detection confidence to consider person

model = YOLO('yolov8n.pt')

csv_file = 'crowd_log.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame Number", "Person Count in Crowd"])

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("❌ Failed to open video.")
    exit()

frame_count = 0
crowd_frame_buffer = deque(maxlen=PERSISTENCE_FRAMES)
debug_frames = []
frame_sample_limit = 5  # Number of sample frames to store for visualization

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read frame.")
        break

    frame_count += 1

    results = model(frame, verbose=False)[0]
    detected_boxes_raw = []

    for box in results.boxes.data.tolist():
        x1, y1, x2, y2, conf, cls = box
        if int(cls) == 0 and conf > CONFIDENCE_THRESHOLD:
            xc = (x1 + x2) / 2
            yc = (y1 + y2) / 2
            w = x2 - x1
            h = y2 - y1
            detected_boxes_raw.append([xc, yc, w, h])

    boxes = [[xc - w / 2, yc - h / 2, xc + w / 2, yc + h / 2] for xc, yc, w, h in detected_boxes_raw]
    centers = [((x1 + x2) / 2, (y1 + y2) / 2) for x1, y1, x2, y2 in boxes]

    if len(centers) < 2:
        print(f"Frame {frame_count}: Not enough people detected ({len(centers)}).")
        crowd_frame_buffer.clear()
        continue

    dist_matrix = distance.cdist(centers, centers)
    close_map = dist_matrix < DISTANCE_THRESHOLD

    groups = []
    visited = set()
    for i in range(len(centers)):
        if i in visited:
            continue
        group = set([i])
        queue = [i]
        while queue:
            current = queue.pop()
            visited.add(current)
            for j in range(len(centers)):
                if j not in visited and close_map[current][j]:
                    group.add(j)
                    queue.append(j)
        if len(group) >= 3:
            groups.append(group)

    if groups:
        largest_crowd_size = max(len(g) for g in groups)
        print(f"[✓] Frame {frame_count}: Crowd detected with {largest_crowd_size} people")
        crowd_frame_buffer.append((frame_count, largest_crowd_size))

        if len(debug_frames) < frame_sample_limit:
            for x1, y1, x2, y2 in boxes:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            debug_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    else:
        print(f"[✗] Frame {frame_count}: No crowd detected")
        crowd_frame_buffer.clear()

    if len(crowd_frame_buffer) == PERSISTENCE_FRAMES:
        logged_frame, logged_count = crowd_frame_buffer[0]
        with open(csv_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([logged_frame, logged_count])
        print(f"[✓] Logged crowd event at frame {logged_frame} with {logged_count} people.")
        crowd_frame_buffer.clear()

cap.release()

for i, frame_img in enumerate(debug_frames):
    plt.figure(figsize=(10, 6))
    plt.imshow(frame_img)
    plt.title(f"Sample Frame {i + 1} with Detections")
    plt.axis('off')
    plt.show()

print("\n=== Logged Crowd Events ===")
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        print(f"Frame {row[0]} → Crowd Size: {row[1]}")


[✓] Frame 1: Crowd detected with 8 people
[✓] Frame 2: Crowd detected with 9 people
[✓] Frame 3: Crowd detected with 9 people
[✓] Logged crowd event at frame 1 with 8 people.
[✓] Frame 4: Crowd detected with 9 people
[✓] Frame 5: Crowd detected with 9 people
[✓] Frame 6: Crowd detected with 9 people
[✓] Logged crowd event at frame 4 with 9 people.
[✓] Frame 7: Crowd detected with 9 people
[✓] Frame 8: Crowd detected with 9 people
[✓] Frame 9: Crowd detected with 9 people
[✓] Logged crowd event at frame 7 with 9 people.
[✓] Frame 10: Crowd detected with 9 people
[✓] Frame 11: Crowd detected with 9 people
[✓] Frame 12: Crowd detected with 9 people
[✓] Logged crowd event at frame 10 with 9 people.
[✓] Frame 13: Crowd detected with 11 people
[✓] Frame 14: Crowd detected with 11 people
[✓] Frame 15: Crowd detected with 6 people
[✓] Logged crowd event at frame 13 with 11 people.
[✓] Frame 16: Crowd detected with 6 people
[✓] Frame 17: Crowd detected with 6 people
[✓] Frame 18: Crowd detected

In [32]:
import csv
csv_file = 'crowd_log.csv'
print("\n=== Logged Crowd Events ===")
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    header = next(reader)
    for row in reader:
        print(f"Frame {row[0]} → Crowd Size: {row[1]}")



=== Logged Crowd Events ===
Frame 1 → Crowd Size: 8
Frame 4 → Crowd Size: 9
Frame 7 → Crowd Size: 9
Frame 10 → Crowd Size: 9
Frame 13 → Crowd Size: 11
Frame 16 → Crowd Size: 6
Frame 19 → Crowd Size: 10
Frame 22 → Crowd Size: 10
Frame 25 → Crowd Size: 8
Frame 28 → Crowd Size: 14
Frame 31 → Crowd Size: 7
Frame 34 → Crowd Size: 13
Frame 37 → Crowd Size: 5
Frame 40 → Crowd Size: 7
Frame 43 → Crowd Size: 6
Frame 46 → Crowd Size: 8
Frame 49 → Crowd Size: 6
Frame 52 → Crowd Size: 6
Frame 55 → Crowd Size: 7
Frame 58 → Crowd Size: 6
Frame 61 → Crowd Size: 6
Frame 64 → Crowd Size: 6
Frame 67 → Crowd Size: 6
Frame 70 → Crowd Size: 6
Frame 73 → Crowd Size: 7
Frame 76 → Crowd Size: 7
Frame 79 → Crowd Size: 7
Frame 82 → Crowd Size: 8
Frame 85 → Crowd Size: 6
Frame 88 → Crowd Size: 4
Frame 91 → Crowd Size: 8
Frame 94 → Crowd Size: 7
Frame 97 → Crowd Size: 7
Frame 100 → Crowd Size: 7
Frame 103 → Crowd Size: 5
Frame 106 → Crowd Size: 6
Frame 109 → Crowd Size: 6
Frame 112 → Crowd Size: 6
Frame 115 → Cr

In [33]:
from google.colab import files
files.download('crowd_log.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>